# Build a Multi-Agent Systems
In this notebook, we'll:
- ✅ Configure the API key to use the Gemini model
- ✅ Learn when to use multi-agent systems in [Agent Development Kit (ADK)](https://google.github.io/adk-docs/)
- ✅ Build a system using an LLM as a "manager"
- ✅ Learn three core workflow patterns (Sequential, Parallel, and Loop) to coordinate the agent teams

## Part 1: Add Gemini API key to the env

This notebook uses the Gemini API, which requires authentication.

1. Get your API key

If you don't have one already, create an API key in Google AI Studio.

2. Add the key to env and load as shown in the script 

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

google_api_key = os.environ.get("GOOGLE_API_KEY")
if not google_api_key:
    print(
        "GOOGLE_API_KEY is not set. Create one in Google AI Studio "
        "(https://aistudio.google.com/) and add it to your env or .env file."
    )
    raise SystemExit("Set GOOGLE_API_KEY and rerun this cell.")

print("✅ GOOGLE_API_KEY loaded.")

✅ GOOGLE_API_KEY loaded.


### validate the api key

In [2]:
from google import genai
from google.genai import types

client = genai.Client(api_key=google_api_key)

try:
    _ = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents="ping",
        config=types.GenerateContentConfig(max_output_tokens=1),
    )
    print("✅ API key validated with Gemini.")
except Exception as err:
    raise SystemExit(f"API key validation failed: {err}")

✅ API key validated with Gemini.


### Import ADK components

In [12]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


## Part 2: Why multi-agent systems? + Design a multi-agent
**The Problem: The "Do-It-All" Agent**
While single agents can accomplish many tasks, they quickly reach limitations as tasks grow in complexity. A monolithic agent that attempts to handle research, writing, editing, and fact-checking simultaneously faces several challenges:
- Bloated and confusing instruction prompts
- Difficulty in debugging (unclear which part failed)
- Maintenance challenges
- Higher risk of producing unreliable results

**The Solution: A Team of Specialists**
Rather than relying on a single agent for every task, a **multi-agent system** employs several specialized agents that work together—just like a collaborative team. Each agent has one clear responsibility (for example, one agent focuses on research, another handles writing). This specialization leads to:
- Easier construction and testing of individual agents
- Greater reliability and maintainability
- Enhanced overall system performance through collaboration

For further details, refer to the [LLM agents in ADK documentation](https://google.github.io/adk-docs/agents/llm-agents/).
**Architecture: Single Agent vs Multi-Agent Team**


### Example: Research + Summarization System
Let's build a system with two specialized agents:

1. **Research Agent** - Searches for information using Google Search
2. **Summarizer Agent** - Creates concise summaries from research findings

### Other Multi-Agent System Examples
<!-- #
# Here are several ideas for multi-agent systems composed of specialized agents:
#
# 1. **Question Answering System**
#    - **Retriever Agent**: Finds relevant documents or passages from a knowledge base.
#    - **Reader Agent**: Extracts and formulates a direct answer from the retrieved text.
#
# 2. **Content Generation Pipeline**
#    - **Research Agent**: Gathers key facts and background information.
#    - **Writer Agent**: Generates a draft article or story based on research.
#    - **Editor Agent**: Refines grammar, style, and coherence.
#
# 3. **Multi-Modal Assistant**
#    - **Image Analysis Agent**: Interprets and describes uploaded images.
#    - **Text Summarization Agent**: Summarizes content from images and user input.
#    - **Fact-Checking Agent**: Verifies the accuracy of generated statements.
#
# 4. **Customer Support System**
#    - **Intent Recognition Agent**: Classifies user requests.
#    - **FAQ Lookup Agent**: Searches a database for relevant answers.
#    - **Escalation Agent**: Determines when to hand off to a human or higher-level agent.
#
# 5. **Creative Design Team**
#    - **Idea Generator Agent**: Proposes new concepts or approaches.
#    - **Critique Agent**: Reviews and suggests improvements. -->

In [4]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings", # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")


✅ research_agent created.


In [5]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model="gemini-2.5-flash-lite",
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")


✅ summarizer_agent created.


Then we bring the agents together under a root agent, or coordinator:

In [6]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model="gemini-2.5-flash-lite",
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[
        AgentTool(research_agent),
        AgentTool(summarizer_agent)
    ],
)

print("✅ root_agent created.")

✅ root_agent created.


Lets run the agent and ask about a topic

In [7]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("What are the latest advancements of India in the field of AI?")


 ### Created new session: debug_session_id

User > What are the latest advancements of India in the field of AI?


ResearchCoordinator > India has been making significant strides in Artificial Intelligence (AI). The government has launched the IndiaAI Mission with a substantial budget to position India as a global AI leader, focusing on accessibility, data enhancement, and industry collaboration. A key development is the emphasis on indigenous AI, including models like BharatGen AI that support 22 Indian languages, and Sarvam AI working towards a sovereign AI ecosystem. India is also expanding its AI integration through Centers of Excellence in critical sectors, promoting state-level AI centers, and incorporating AI into governance for better efficiency and service delivery. Furthermore, the country boasts a dynamic AI startup scene and a strong talent pool, ranking highly in global AI skills and research participation.


Congratulations! We've just created your first multi-agent system—a setup where a single "coordinator" agent orchestrates a workflow by managing sub-agents. This pattern is both powerful and flexible for many use cases.

‼️ However, relying on an LLM's instructions to control the order can sometimes be unpredictable. Next, we'll explore a different pattern that gives you guaranteed, step-by-step execution.

## Part 3 Sequential Workflows - The Assembly Line

**The Problem: Unpredictable Order**

The previous multi-agent system worked, but it relied on a **detailed instruction prompt** to force the LLM to run steps in order. This can be unreliable. A complex LLM might decide to skip a step, run them in the wrong order, or get "stuck," making the process unpredictable.

**The Solution: A Fixed Pipeline**

When you need tasks to happen in a **guaranteed, specific order**, you can use a `SequentialAgent`. This agent acts like an assembly line, running each sub-agent in the exact order you list them. The output of one agent automatically becomes the input for the next, creating a predictable and reliable workflow.

**Use Sequential when:** Order matters, you need a linear pipeline, or each step builds on the previous one.

To learn more, check out the documentation related to [sequential agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/).

### Example: Blog Post Creation with Sequential Agents

Let's build a system with three specialized agents:

1. **Outline Agent** - Creates a blog outline for a given topic
2. **Writer Agent** - Writes a blog post
3. **Editor Agent** - Edits a blog post draft for clarity and structure

In [8]:
# Outline Agent: Creates the initial blog post outline.
outline_agent = Agent(
    name="OutlineAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline", # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [ ]:
# Writer Agent: Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft", # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [ ]:
# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog", # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


Then we bring the agents together under a sequential agent, which runs the agents in the order that they are listed:

In [13]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [14]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a blog post about the role of Indian government in AI.")


 ### Created new session: debug_session_id

User > Write a blog post about the role of Indian government in AI.
OutlineAgent > ## Blog Post Outline: The Indian Government's AI Ascendancy

**Headline:** India's AI Ambitions: How the Government is Shaping the Future of Artificial Intelligence

**Introduction Hook:** From smart cities to digital healthcare, Artificial Intelligence is no longer a futuristic fantasy in India – it's a rapidly unfolding reality. But what role is the Indian government playing in this technological revolution? Dive into how policy, investment, and a vision for "AI for All" are propelling India onto the global AI stage.

---

**Main Section 1: Policy & Vision – Laying the AI Groundwork**

*   **National Strategy for AI:** Explore the government's overarching strategy, emphasizing its goals for economic growth, social development, and making India a global leader in AI research and application.
*   **Ethical AI Frameworks:** Discuss the government's focus on res

---
## Part 4 Parallel Workflows - Independent Researchers

**The Problem: The Bottleneck**

The previous sequential agent is great, but it's an assembly line. Each step must wait for the previous one to finish. What if you have several tasks that are **not dependent** on each other? For example, researching three *different* topics. Running them in sequence would be slow and inefficient, creating a bottleneck where each task waits unnecessarily.

**The Solution: Concurrent Execution**

When you have independent tasks, you can run them all at the same time using a `ParallelAgent`. This agent executes all of its sub-agents concurrently, dramatically speeding up the workflow. Once all parallel tasks are complete, you can then pass their combined results to a final 'aggregator' step.

**Use Parallel when:** Tasks are independent, speed matters, and you can execute concurrently.

To learn more, check out the documentation related to [parallel agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/).


### Example: Parallel Multi-Topic Research

Let's build a system with four agents:

1. **Tech Researcher** - Researches AI/ML news and trends
2. **Health Researcher** - Researches recent medical news and trends
3. **Finance Researcher** - Researches finance and fintech news and trends
4. **Aggregator Agent** - Combines all research findings into a single summary

In [15]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research", # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [16]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research", # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [17]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research", # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [18]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model="gemini-2.5-flash-lite",
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary", # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [19]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [21]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Compare India and China on Tech, Health, and Finance")


 ### Created new session: debug_session_id

User > Compare India and China on Tech, Health, and Finance
FinanceResearcher > Here's a comparison of India and China across Technology, Health, and Finance:

**Technology:**

*   **India:** A global IT services and outsourcing hub, with a burgeoning startup ecosystem and strong growth in AI, cloud computing, and cybersecurity. The government's "Digital India" initiative further fuels growth and digital literacy.
*   **China:** A tech powerhouse with dominant players in e-commerce, social media, AI, and telecommunications. China is a leader in emerging technologies like 5G and IoT, with massive data utilization driving AI development.

**Health:**

*   **India:** A rapidly growing healthcare sector, driven by increasing demand, private sector investment, and a focus on medical tourism and telemedicine. Government spending is also increasing, aiming to improve accessibility and quality.
*   **China:** The world's second-largest healthcare ma

## Part 5  Loop Workflows - The Refinement Cycle

**The Problem: One-Shot Quality**

All the workflows we've seen so far run from start to finish. The `SequentialAgent` and `ParallelAgent` produce their final output and then stop. This 'one-shot' approach isn't good for tasks that require refinement and quality control. What if the first draft of our story is bad? We have no way to review it and ask for a rewrite.

**The Solution: Iterative Refinement**

When a task needs to be improved through cycles of feedback and revision, you can use a `LoopAgent`. A `LoopAgent` runs a set of sub-agents repeatedly *until a specific condition is met or a maximum number of iterations is reached.* This creates a refinement cycle, allowing the agent system to improve its own work over and over.

**Use Loop when:** Iterative improvement is needed, quality refinement matters, or you need repeated cycles.

To learn more, check out the documentation related to [loop agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/).

### Example: Iterative Story Refinement

Let's build a system with two agents:

1. **Writer Agent** - Writes a draft of a short story
2. **Critic Agent** - Reviews and critiques the short story to suggest improvements

In [22]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story", # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [23]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique", # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


Now, we need a way for the loop to actually stop based on the critic's feedback. The `LoopAgent` itself doesn't automatically know that "APPROVED" means "stop."

We need an agent to give it an explicit signal to terminate the loop.

We do this in two parts:

1. A simple Python function that the `LoopAgent` understands as an "exit" signal.
2. An agent that can call that function when the right condition is met.

First, you'll define the `exit_loop` function:

In [25]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

print("✅ exit_loop function created.")

✅ exit_loop function created.


In [26]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    
    output_key="current_story", # It overwrites the story with the new, refined version.
    tools=[FunctionTool(exit_loop)], # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


Then we bring the agents together under a loop agent, which is itself nested inside of a sequential agent.

This design ensures that the system first produces an initial story draft, then the refinement loop runs up to the specified number of `max_iterations`:

In [27]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2, # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [29]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a short story about a lighthouse keeper who discovers a mysterious, glowing map")


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map


_ResourceExhaustedError: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 51.145040302s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_co